【問題1】学習と推定

【問題2】コードリーディング

#%% md

# Sprint17 Faster R-CNNとyolo-V3

#%% md

# Faster R-CNN

#%% md

# 問題1

## プログラム準備

https://github.com/duckrabbits/ObjectDetection/tree/master

上記で取得してきたフォルダの中に、このSprint17-rcnn.ipynbを配置します。

## データ準備
+ Kaggleからダウンロードしたthe-simpsons-characters-dataset.zipを解凍します。
+ simpsons_dataset.zipを解凍します。
+ 解凍したsimpsons_datasetフォルダをプロジェクトフォルダ配下に格納します。
+ annotation.txtもプロジェクトフォルダ配下に格納します。 （kaggleからダウンロードしたannotation.txtではなく、ここにあるannotation.txtを使いましょう）

## データ確認

このSprint17-rcnn.ipynbがあるフォルダ内は下記のようになっていると思います。

- kaggle_simpson_testset /（ディレクトリ）
- simpsons_dataset /（ディレクトリ）
- model /（ディレクトリ）
- README.md（ファイル）
- annotation.txt（ファイル）
- predict.py（ファイル）
- train.py（ファイル）
- Sprint17-rcnn.ipynb（ファイル）

#%% md

## 学習

train.py内の流れとしては、modelフォルダ内の各種プログラムを参照し、モデルを構築し、annotation.txt内の記述に従って、学習用データを取得し学習を実行している

#%%

!python train.py -p annotation.txt

#%% md

## 推測

`-c`のオプションのファイル名は自身の学習実行時に保存されたファイル名を指定してやる必要があります。

#%%

!python predict.py -i kaggle_simpson_testset/kaggle_simpson_testset -c ./save/train_20210830-170538_config.pickle

In [2]:
python train.py -p annotation.txt

SyntaxError: invalid syntax (<ipython-input-2-a88aa3fba454>, line 1)

In [ ]:
#%% md

# 問題2

下記一例です。。。。

### RPNは、`resnet.py`の下記で実装されている。

```def rpn(base_layers,num_anchors)```

### ROIプーリングは、`RoiPoolingConv.py`の下記で実装されている。

```class RoiPoolingConv(Layer)```

### 各種損失計算は、`losses.py`の下記で実装されている。

```
def rpn_loss_regr(num_anchors)
def rpn_loss_cls(num_anchors)
def class_loss_regr(num_classes)
def class_loss_cls(y_true, y_pred)
```

In [ ]:
## プログラム準備
https://github.com/qqwweee/keras-yolo3

上記で取得してきたフォルダの中に、このSprint17-yolo.ipynbを配置します。

## データ準備
faster r cnn で取得してきたデータをこのSprint17-yolo.ipynbと同階層に設置します。

## 学習済みモデルの取得

下記にアクセスし、学習済みモデルを取得してきます。

https://pjreddie.com/media/files/yolov3.weights

## データ確認

このSprint17-yolo.ipynbがあるフォルダ内は下記のようになっていると思います。

- 上記リポジトリのファイルやディレクトリ群（ファイルやディレクトリ）
- kaggle_simpson_testset /（ディレクトリ）
- simpsons_dataset /（ディレクトリ）
- Sprint17-rcnn.ipynb（ファイル）
- yolov3.weights

#%% md

## モデル形式の変更

#%%

!python convert.py yolov3.cfg yolov3.weights model_data/yolo_weights.h5

In [1]:
#学習
%run train.py -p annotation.txt

Exception: File `'train.py'` not found.

In [ ]:
#%% md

# 問題3　学習済みの重みによる推定

#%%

!python yolo_video.py --image

In [ ]:
#%% md

# 問題4　学習のためのファイルを作成

annotation.txtのファイル形式がfaster-rcnnの形式と違うため、変更したものを作成する

rcnnの際に使ったannotation.txtをこのフォルダにコピーし、annotation_rcnn.txtとrenameしてください。

#%%

# ライブラリのimport
import pandas as pd
from sklearn.preprocessing import LabelEncoder

#%%

# 読み込みと確認
annotation_df = pd.read_csv('annotation_rcnn.txt',header=None)
n_sample, n_col = annotation_df.shape
annotation_df.head()

#%%

# キャラクター名を数値（クラス・ラベル）に変更する
le = LabelEncoder()
annotation_df.iloc[:,5] = le.fit_transform(annotation_df.iloc[:,5])
annotation_df.head()

#%%

rcnn_path = 'annotation_rcnn.txt'
yolo_path = 'annotation.txt'

# rcnnの方のannotation.txtを開く
with open(rcnn_path) as f:
    # データを読み込む
    lines = f.readline() 
    # データ数（行数）でループ
    for i in range(n_sample):
        # カンマ区切りのデータをリストに変換
        split_line = lines.split(',') 
        # 画像のパスは一番前に格納されているので取得
        image_path = split_line[0]
        # パス変更
        split_line[0] = './' + image_path
        # 末尾をキャラクター名から数値（クラス・ラベル）に変更→改行コード加える
        split_line[-1] = str(annotation_df.iloc[i,5]) + '\n'  
        # yoloの方のannotation.txtを開き保存
        with open(yolo_path, mode='a') as out_f:
            join_line = ','.join(split_line)  
            join_line = join_line.replace('.jpg,','.jpg ')  
            out_f.write(join_line)

In [ ]:
#%% md

# 問題5　学習が行えることの確認


問題4で作成したファイルを使用して学習してください。実行環境で学習に時間がかかる場合は、学習が行えることを確認するのみで終えて構いません。


このサンプルコードのやり方でいくと、train.py 17行目 `annotation_path = 'train.txt'`を`annotation_path = 'annotation.txt'`に変えてやる必要があります。

#%%

!python train.py